# Introduction

In [1]:
%ls

Faster1.ipynb               pi/
algorithms/                 profiling.pdf
caching/                    profiling_presentation.pdf
measuring/


In [3]:
%less Faster1.ipynb

In [4]:
ls

Faster1.ipynb               pi/
algorithms/                 profiling.pdf
caching/                    profiling_presentation.pdf
measuring/


In [5]:
cd measuring/

/Users/mike/pycon2017/measuring


In [6]:
ls

accumulate.py                    memory_size_pympler.py
calc.py                          profile_me.py
clock_check.py                   profile_me_use_line_profiler.py
cpu_time.py                      profile_speed.py
create_list.py                   pympler_list_growth.py
list_alloc_steps.py              pystone2.py
local_ref.py                     pystone2_3.py
make_list_gen.py                 pystone3.py
memory_growth_hpy.py             pystone_converter.py
memory_growth_pympler.py         use_mem.py
memory_size_hpy.py


# Profiling CPU

In [8]:
# %load profile_me.py
# file profile_me.py

"""Example to be profiled.
"""
import sys
import time

if sys.version_info.major < 3:
    range = xrange


def fast():
    """Wait 0.001 seconds.
    """
    time.sleep(1e-3)


def slow():
    """Wait 0.1 seconds.
    """
    time.sleep(0.1)


def use_fast():
    """Call `fast` 100 times.
    """
    for _ in range(100):
        fast()


def use_slow():
    """Call `slow` 100 times.
    """
    for _ in range(100):
        slow()


if __name__ == '__main__':
    use_fast()
    use_slow()


In [9]:
import cProfile

In [10]:
profile = cProfile.Profile()

In [11]:
profile.runcall(use_fast)

In [12]:
profile.print_stats()

         202 function calls in 0.127 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.000    0.000    0.126    0.001 <ipython-input-8-31b8b4df29c5>:13(fast)
        1    0.000    0.000    0.127    0.127 <ipython-input-8-31b8b4df29c5>:25(use_fast)
      100    0.126    0.001    0.126    0.001 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [13]:
p2 = cProfile.Profile()

In [14]:
p2.runcall(use_slow)

In [15]:
p2.print_stats()

         202 function calls in 10.222 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.001    0.000   10.221    0.102 <ipython-input-8-31b8b4df29c5>:19(slow)
        1    0.001    0.001   10.222   10.222 <ipython-input-8-31b8b4df29c5>:32(use_slow)
      100   10.220    0.102   10.220    0.102 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [16]:
cProfile.run('use_fast()')

         204 function calls in 0.126 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.001    0.000    0.126    0.001 <ipython-input-8-31b8b4df29c5>:13(fast)
        1    0.000    0.000    0.126    0.126 <ipython-input-8-31b8b4df29c5>:25(use_fast)
        1    0.000    0.000    0.126    0.126 <string>:1(<module>)
        1    0.000    0.000    0.126    0.126 {built-in method builtins.exec}
      100    0.125    0.001    0.125    0.001 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [17]:
use_fast.__name__

'use_fast'

In [18]:
cProfile.run('use_fast()', 'fast.stats')

In [19]:
ls *.stats

fast.stats


In [20]:
import pstats

In [21]:
stats = pstats.Stats('fast.stats')

In [22]:
stats.print_stats()

Thu May 18 18:44:23 2017    fast.stats

         204 function calls in 0.125 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.125    0.125 {built-in method builtins.exec}
      100    0.124    0.001    0.124    0.001 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
      100    0.000    0.000    0.125    0.001 <ipython-input-8-31b8b4df29c5>:13(fast)
        1    0.000    0.000    0.125    0.125 <ipython-input-8-31b8b4df29c5>:25(use_fast)
        1    0.000    0.000    0.125    0.125 <string>:1(<module>)




In [24]:
stats.sort_stats('time').print_stats()

Thu May 18 18:44:23 2017    fast.stats

         204 function calls in 0.125 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.124    0.001    0.124    0.001 {built-in method time.sleep}
      100    0.000    0.000    0.125    0.001 <ipython-input-8-31b8b4df29c5>:13(fast)
        1    0.000    0.000    0.125    0.125 <ipython-input-8-31b8b4df29c5>:25(use_fast)
        1    0.000    0.000    0.125    0.125 {built-in method builtins.exec}
        1    0.000    0.000    0.125    0.125 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [25]:
stats.sort_stats('call').print_stats()

Thu May 18 18:44:23 2017    fast.stats

         204 function calls in 0.125 seconds

   Ordered by: call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.124    0.001    0.124    0.001 {built-in method time.sleep}
      100    0.000    0.000    0.125    0.001 <ipython-input-8-31b8b4df29c5>:13(fast)
        1    0.000    0.000    0.125    0.125 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.125    0.125 <ipython-input-8-31b8b4df29c5>:25(use_fast)
        1    0.000    0.000    0.125    0.125 <string>:1(<module>)




In [26]:
stats.print_callers('fast')

   Ordered by: call count
   List reduced from 6 to 2 due to restriction <'fast'>

Function                                     was called by...
                                                 ncalls  tottime  cumtime
<ipython-input-8-31b8b4df29c5>:13(fast)      <-     100    0.000    0.125  <ipython-input-8-31b8b4df29c5>:25(use_fast)
<ipython-input-8-31b8b4df29c5>:25(use_fast)  <-       1    0.000    0.125  <string>:1(<module>)




In [27]:
stats.print_callees('use_fast')

   Ordered by: call count
   List reduced from 6 to 1 due to restriction <'use_fast'>

Function                                     called...
                                                 ncalls  tottime  cumtime
<ipython-input-8-31b8b4df29c5>:25(use_fast)  ->     100    0.000    0.125  <ipython-input-8-31b8b4df29c5>:13(fast)




In [28]:
%prun use_fast()

In [30]:
%prun?

In [38]:
%prun -q  -T fast.txt  use_fast()

 
*** Profile printout saved to text file 'fast.txt'. 


In [40]:
# %load clock_check.py
# file: measuring/clock_check.py

"""Checking different timing functions.
"""

from __future__ import print_function

import os
import sys
import time
import timeit


if sys.version_info.major < 3:
    range = xrange


def clock_check(duration=1):
    """Check the measured time with different methods.
    """
    start_os_time0 = os.times()[0]
    start_time_clock = time.clock()
    start_default_timer = timeit.default_timer()
    for _ in range(int(1e6)):
        1 + 1
    time.sleep(duration)
    durtation_os_time0 = os.times()[0] - start_os_time0
    durtation_time_clock = time.clock() - start_time_clock
    durtation_default_timer = timeit.default_timer() - start_default_timer
    print('durtation_os_time0:     ', durtation_os_time0)
    print('durtation_time_clock:   ', durtation_time_clock)
    print('durtation_default_timer:', durtation_default_timer)


if __name__ == '__main__':
    clock_check()


durtation_os_time0:      0.030000000000000027
durtation_time_clock:    0.031876000000000015
durtation_default_timer: 1.0336465429863892


In [41]:
timeit.default_timer

<function time.perf_counter>

In [43]:
# %load cpu_time.py
# file: cpu_time.py

"""Measuring CPU time instead of wall clock time.
"""

import cProfile
import os
import sys
import time

# Make it work with Python 2 and Python 3.
if sys.version_info.major < 3:
    range = xrange


def cpu_time():
    """Function for cpu time. Os dependent.
    """
    if sys.platform == 'win32':
        return os.times()[0]
    else:
        return time.clock()


def sleep():
    """Wait 2 seconds.
    """
    time.sleep(2)


def count():
    """100 million loops.
    """
    for _ in range(int(1e8)):
        1 + 1


def test():
    """Run functions
    """
    sleep()
    count()


def profile():
    """Profile with wall clock and cpu time.
    """
    profiler = cProfile.Profile()
    profiler.run('test()')
    profiler.print_stats()

    profiler = cProfile.Profile(cpu_time)
    profiler.run('test()')
    profiler.print_stats()

if __name__ == '__main__':
    profile()


         7 function calls in 5.311 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.002    2.002 <ipython-input-43-d2afec7be092>:26(sleep)
        1    3.309    3.309    3.309    3.309 <ipython-input-43-d2afec7be092>:32(count)
        1    0.000    0.000    5.311    5.311 <ipython-input-43-d2afec7be092>:39(test)
        1    0.000    0.000    5.311    5.311 <string>:1(<module>)
        1    0.000    0.000    5.311    5.311 {built-in method builtins.exec}
        1    2.002    2.002    2.002    2.002 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         7 function calls in 3.132 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 <ipython-input-43-d2afec7be092>:26(sleep)
        1    3.130    3.130    3.130    

In [44]:
%timeit 1 + 1

15.2 ns ± 0.275 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [45]:
import time

In [52]:
%%timeit -n 1 -r 1

for _ in range(10**3):
    time.sleep(1e-3)

1.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [47]:
%timeit?

In [54]:
cd ../pi/

/Users/mike/pycon2017/pi


In [55]:
ls

numpy_pi.py   simple_pi.py


In [58]:
# %load simple_pi.py
# file: simple_pi.py

"""Calculating pi with Monte Carlo.
"""

from __future__ import print_function

import math
import random
import sys


if sys.version_info[0] < 3:
    range = xrange


def pi_plain(total):
    """Calculate pi with `total` hits.
    """
    count_inside = 0
    for _ in range(total):
        x = random.random()
        y = random.random()
        dist = math.sqrt(x * x + y * y)
        if dist < 1:
            count_inside += 1
    return 4.0 * count_inside / total

if __name__ == '__main__':

    def test():
        """Check if it works.
        """
        n = int(1e7)
        print('pi:', pi_plain(n))

    test()


pi: 3.140898


In [61]:
# %load numpy_pi.py
# file: numpy_pi.py
"""Calculating pi with Monte Carlo Method and NumPy.
"""

from __future__ import print_function

import numpy                                                   #1


def pi_numpy(total):                                           #2
    """Compute pi.
    """
    x = numpy.random.rand(total)                               #3
    y = numpy.random.rand(total)                               #4
    dist = numpy.sqrt(x * x + y * y)                           #5
    count_inside = len(dist[dist < 1])                         #6
    return 4.0 * count_inside / total

if __name__ == '__main__':

    def test():
        """Time the execution.
        """
        import timeit
        start = timeit.default_timer()
        pi_numpy(int(1e6))
        print('run time', timeit.default_timer() - start)
    test()


run time 0.05484786001034081


In [62]:
n = int(1e6)

In [63]:
%timeit pi_plain(n)

498 ms ± 15.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
%timeit pi_numpy(n)

41 ms ± 840 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
%load_ext snakeviz

In [66]:
%snakeviz pi_numpy(n)

 
*** Profile stats marshalled to file '/var/folders/p_/yckgs9wn0xbcmqc92zk3z4240000gn/T/tmpis8k3f1k'. 


In [67]:
cd ../measuring/

/Users/mike/pycon2017/measuring


In [68]:
ls

accumulate.py                    memory_size_hpy.py
calc.py                          memory_size_pympler.py
clock_check.py                   profile_me.py
cpu_time.py                      profile_me.py.prof
create_list.py                   profile_me_use_line_profiler.py
fast.stats                       profile_speed.py
fast.txt                         pympler_list_growth.py
list_alloc_steps.py              pystone2.py
local_ref.py                     pystone2_3.py
make_list_gen.py                 pystone3.py
memory_growth_hpy.py             pystone_converter.py
memory_growth_pympler.py         use_mem.py


In [ ]:
# %load profile_me_use_line_profiler.py
# file profile_me_use_line_profiler.py

"""Example to be profiled.
"""

import time
import sys

if sys.version_info.major < 3:
    range = xrange


def fast():
    """Wait 0.001 seconds.
    """
    time.sleep(1e-3)


def slow():
    """Wait 0.1 seconds.
    """
    time.sleep(0.1)

@profile
def use_fast():
    """Call `fast` 100 times.
    """
    for _ in range(100):
        fast()

@profile
def use_slow():
    """Call `slow` 100 times.
    """
    for _ in range(100):
        slow()


if __name__ == '__main__':
    use_fast()
    use_slow()


In [ ]:
# %load accumulate.py
# file accumulate.py

"""Simple test function for line_profiler.
"""

@profile
def accumulate(iterable):
    """Accumulate the intermediate steps in summing all elements.

    The result is a list with the length of `iterable`.
    The last element is the sum of all elements of `iterable`
    >>> accumulate(range(5))
    [0, 1, 3, 6, 10]
    >>> accumulate(range(10))
    [0, 1, 3, 6, 10, 15, 21, 28, 36, 45]
    """
    acm = [iterable[0]]
    for elem in iterable[1:]:
        old_value = acm[-1]
        new_value = old_value + elem
        acm.append(new_value)
    return acm


if __name__ == '__main__':
    accumulate(range(10))
    accumulate(range(100))


In [ ]:
# %load calc.py
#calc.py

"""Simple test function for line_profiler doing some math.
"""

import math
import sys

if sys.version_info.major < 3:
    range = xrange


@profile
def calc(number, loops=1000):
    """Do some math calculations.
    """
    sqrt = math.sqrt
    for x in range(loops):
        x = number + 10
        x = number * 10
        x = number ** 10
        x = pow(number, 10)
        x = math.sqrt(number)
        x = sqrt(number)
        math.sqrt
        sqrt

if __name__ == '__main__':
    calc(100, int(1e5))


In [72]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [73]:
%lprun -f use_fast -f fast use_fast()

In [75]:
# %load create_list.py
# file: create_list.py

import sys

if sys.version_info.major < 3:
    range = xrange


def insert_zero(n=int(1e4)):
    """Assemble list with `insert`. Inefficient.
    """
    L = []
    for x in range(n):
        L.insert(0, x)
    return L


def append_reverse(n=int(1e4)):
    """Assemble list with `append` and `reverse`.
    """
    L = []
    for x in range(n):
        L.append(x)
    L.reverse()
    return L


In [78]:
%timeit insert_zero(int(2e4))

87.6 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [79]:
%timeit append_reverse(int(2e4))

2.16 ms ± 30.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [80]:
%lprun -f insert_zero insert_zero()

In [81]:
%lprun -f append_reverse append_reverse()